# Gmailを取得する

## 環境構築

* Rubyは2.3系

### Gemfile

```
$ vi Gemfile
================================================
# frozen_string_literal: true

source "https://rubygems.org"

git_source(:github) {|repo_name| "https://github.com/#{repo_name}" }

gem "google-api-client"
gem 'debase', '0.2.2.beta8'
gem 'ruby-debug-ide', '0.6.1.beta9'
================================================

$ bundle install --path vendor/bundle
```

### アカウント情報準備

* https://developers.google.com/gmail/api/quickstart/ruby
    * jsonファイルをダウンロードしてclient_secret.jsonにする
    * 「OAuth 2.0 クライアント ID」からクライアントIDとクライアントシークレットをコピペで控える
    * quickstart.rbをコピペで準備

<img src="img/secret.png" alt="情報" title="情報" width="600" height="300" align="left" />  

<br clear="left">

### googleauth修正(非推奨)

* そのままでは使えなかったので情報をコードに直接埋め込んだ

```
$ vi vendor/bundle/ruby/2.3.0/gems/googleauth-0.6.2/lib/googleauth/client_id.rb
================================================
■ before
      def self.from_hash(config)
        raise 'Hash can not be nil.' if config.nil?
        raw_detail = config[INSTALLED_APP] || config[WEB_APP]
        raise MISSING_TOP_LEVEL_ELEMENT_ERROR if raw_detail.nil?
        ClientId.new(raw_detail[CLIENT_ID], raw_detail[CLIENT_SECRET])
      end

■ after
      def self.from_hash(config)
        raise 'Hash can not be nil.' if config.nil?
        #raw_detail = config[INSTALLED_APP] || config[WEB_APP]
        #raise MISSING_TOP_LEVEL_ELEMENT_ERROR if raw_detail.nil?
        #ClientId.new(raw_detail[CLIENT_ID], raw_detail[CLIENT_SECRET])
        ClientId.new("123456789123-abc1d2efghif3klm456no78pq9rstuv0.apps.googleusercontent.com", "ABCDEfgHIFk1lmNOP2qRstu3")
      end
================================================
```

* 参考
    * [rubyでの正しいgmailの取得方法を教える](https://qiita.com/hatappi/items/8d701c934b143e490599)

例quickstart.rbに書き込む

```
# How to use Gmail API
# https://qiita.com/hatappi/items/8d701c934b143e490599
# Class: Google::Apis::GmailV1::GmailService
# http://www.rubydoc.info/github/google/google-api-ruby-client/Google/Apis/GmailV1/GmailService

### get message id
user_id = 'me'
# result = service.list_user_messages(user_id)
result = service.list_user_messages(user_id, q: "from:info@hoge.example.co.jp")
message_ids = result.messages.map { |message| message.id }

### get messages
messages = []
message_ids.each do |id|
  messages << service.get_user_message(user_id, id)
end

puts messages[4].snippet
```

正規表現練習

```
0> messages[4].payload.body.data.match(%r{("http)://pmrd(.*?")})[0]
=> "\"http://hoge.example.co.jp/?r=AbC1DeF2gHI%3J&k=LMN4OP5&u=6QRST7UV\""
```
